<a href="https://colab.research.google.com/github/enginpaksoy/ML/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
class KNN:
    def __init__(self, k):
        """
        Initialize the KNN class with k neighbors.
        Args:
            k (int): Number of nearest neighbors.
        """
        self.k = k
        if(k % 2 != 1):
            raise ValueError("k must be an odd number")

    def fit(self, training_features, training_labels):
        """
        Store the training data.
        Args:
            training_features (numpy array): Features of the training data.
            training_labels (numpy array): Labels of the training data.
        """
        self.training_features = training_features
        self.training_labels = training_labels

    def predict(self, test_features):
        """
        Predict the labels for the test data.
        Args:
            test_features (numpy array): Features of the test data.
        Returns:
            numpy array: Predicted labels for the test data.
        """
        predictions = []
        for test_instance in test_features:
            distances = []
            # Calculate distance from the test instance to all training instances
            for training_instance in self.training_features:
                distances.append(self.euclidean_distance(test_instance, training_instance))

            # Find the k nearest neighbors
            nearest_neighbors = np.argsort(distances)[:self.k]
            # records the indices with the lowest distance between them

            nearest_labels = [self.training_labels[i] for i in nearest_neighbors]
            # nearest_neighbors contains indices of the nearest neighbors
            # Create a list of labels for these nearest neighbors by looking up the indices in self.training_labels

            # Determine the most common label (voting)
            predicted_label = max(set(nearest_labels), key=nearest_labels.count)
            predictions.append(predicted_label)

        return np.array(predictions)

    def train_test_split(self, data, test_size=0.3):
        """
        Split the data into training and test sets.
        Args:
            data (numpy array): Data to be split.
            test_size (float): Size of the test set.
        Returns:
            tuple: Training and test sets.
        """
        num_samples = data.shape[0]  # Gets the number of samples in the data array.
        num_test_samples = int(num_samples * test_size)  # Determines the size of the test set.
        indices = np.random.permutation(num_samples)  # Creates an array of length num_samples and randomizes the elements of this array.
        test_indices = indices[:num_test_samples]  # Takes the first n(length num_samples) indices for the test set.
        train_indices = indices[num_test_samples:]  # Takes the remaining indices for the training set.
        self.test_indices = test_indices
        self.train_indices = train_indices
        return data[train_indices], data[test_indices]  # Returns the training and test sets.

    def euclidean_distance(self, instance1, instance2):
        """
        Calculate the Euclidean distance between two instances.
        Args:
            instance1 (numpy array): The first instance.
            instance2 (numpy array): The second instance.
        Returns:
            float: The Euclidean distance between the two instances.
        """
        instance1_new = np.array(instance1[:-1])
        instance2_new = np.array(instance2[:-1])
        # The code creates new arrays from instance1 and instance2, excluding their last elements.

        return np.sqrt(np.sum((instance1_new - instance2_new) ** 2))


In [ ]:
data = pd.read_csv('/content/sample_data/iris.csv', sep=',', skiprows=0)
data_array = data.to_numpy()

# Create a KNN model with k=5 neighbors
knn = KNN(k=5)

# Seperating training data and testing data
# 30% of the data is reserved for testing, 70% is reserved for training
train_set, test_set = knn.train_test_split(data_array, test_size=0.3)

# Separate features and labels
# [:, :-1]  # All rows, all columns except the last
# [:, -1]   # All rows, only the last column
train_features, train_labels = train_set[:, :-1], train_set[:, -1]
test_features, test_labels = test_set[:, :-1], test_set[:, -1]

# Train the model
knn.fit(train_features, train_labels)

# Predict the labels for the test set
predictions = knn.predict(test_features)

# Evaluate the accuracy
accuracy = np.mean(predictions == test_labels)
# If the value is equal it gives 1, if not it gives 0. and creates an array from this data and takes the average of this array

print(f'# Accuracy: {accuracy:.2f}', end = "\n\n")

# Which indices are reserved for training
print("# Train Indices: ", np.sort(knn.train_indices), end = "\n\n")

# Which indices are reserved for testing
print("# Test Indices: ", np.sort(knn.test_indices), end = "\n\n")

# Different predictions from the real ones in the test data
print("# Different predictions from the real ones in the test data: ", end="\n")
for i in range(test_features.shape[0]):
    if(test_labels[i] != predictions[i]):
        print(f'Indices: {knn.test_indices[i]}, Test Labels: {test_labels[i]}, Predictions: {predictions[i]}', end = "\n")

In [ ]:
knnn = {}
predictions = {}
accuracy = {}

for i in range(1, 149, 2):
    knnn[i] = KNN(k=i)
    knnn[i].fit(train_features, train_labels)
    predictions[i] = knnn[i].predict(test_features)
    accuracy[i] = np.mean(predictions[i] == test_labels)
    print(f'# Accuracy for k = {i}: {accuracy[i]:.2f}')